In [4]:
import pandas as pd
import numpy as np

In [2]:
# Load the dataset
df = pd.read_csv("dirty_cafe_sales.csv")

In [3]:
# Show the first few rows
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
# Summary info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [8]:
# Check for missing values
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [11]:
# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [13]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [14]:
# Replace strings with ERROR, UNKNOWN & (blanks) with NaNs - Standardising invalid values
df.replace(["ERROR", "UNKNOWN", ""], np.nan, inplace=True)

In [16]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [20]:
# Checking data types for each column
df.dtypes

transaction_id       object
item                 object
quantity            float64
price_per_unit       object
total_spent          object
payment_method       object
location             object
transaction_date     object
dtype: object

In [21]:
# Convert columns to correct types
# Convert quantity, price_per_unit & total_spent to numeric values
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')
df['total_spent'] = pd.to_numeric(df['total_spent'], errors='coerce')

In [22]:
df.dtypes

transaction_id       object
item                 object
quantity            float64
price_per_unit      float64
total_spent         float64
payment_method       object
location             object
transaction_date     object
dtype: object

In [23]:
# Convert transaction_date to datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

In [24]:
df.dtypes

transaction_id              object
item                        object
quantity                   float64
price_per_unit             float64
total_spent                float64
payment_method              object
location                    object
transaction_date    datetime64[ns]
dtype: object

In [28]:
# Fill numeric missing values with median
df['quantity'] = df['quantity'].fillna(df['quantity'].median())
df['price_per_unit'] = df['price_per_unit'].fillna(df['price_per_unit'].median())
df['total_spent'] = df['quantity'] * df['price_per_unit']

In [31]:
# Fill categorical missing values with 'Unknown'
df['item'] = df['item'].fillna('Unknown')
df['payment_method'] = df['payment_method'].fillna('Unknown')
df['location'] = df['location'].fillna('Unknown')

In [32]:
# Fill missing dates with the mode (most common date)
df['transaction_date'] = df['transaction_date'].fillna(df['transaction_date'].mode()[0])

In [33]:
df.isnull().sum()

transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64

In [34]:
# Create new columns such as Day of the Week and Transaction Month
df['day_of_week'] = df['transaction_date'].dt.day_name()
df['month'] = df['transaction_date'].dt.month_name()

In [35]:
df[['transaction_date', 'day_of_week', 'month']].head()

,transaction_date,day_of_week,month
0,2023-09-08,Friday,September
1,2023-05-16,Tuesday,May
2,2023-07-19,Wednesday,July
3,2023-04-27,Thursday,April
4,2023-06-11,Sunday,June


In [36]:
# Check for duplicates
df.duplicated().sum()

0

In [37]:
# Confirm all total_spent values are correct
(df['total_spent'] == df['quantity'] * df['price_per_unit']).all()

True

In [38]:
# Most popular items
df['item'].value_counts().head()

item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Name: count, dtype: int64

In [39]:
# Payment method usage
df['payment_method'].value_counts()

payment_method
Unknown           3178
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64

In [40]:
# Sales by weekday
df.groupby('day_of_week')['total_spent'].sum().sort_values(ascending=False)

day_of_week
Monday       16438.5
Sunday       12432.0
Thursday     12394.0
Friday       12348.5
Saturday     12040.5
Tuesday      12040.5
Wednesday    11787.5
Name: total_spent, dtype: float64

In [41]:
# Monthly sales trend
df.groupby('month')['total_spent'].sum()

month
April         7193.5
August        7101.0
December      7221.5
February     10908.0
January       7258.0
July          6927.5
June          7417.5
March         7215.5
May           6978.5
November      7021.0
October       7353.5
September     6886.0
Name: total_spent, dtype: float64

In [42]:
df.to_csv("cleaned_cafe_sales.csv", index=False)